In [131]:
# creating the necessary imports

import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
from math import radians

In [135]:
# Defining the ideal circuit to generate Bell State: |01> + |10>

theta = Parameter('param1')

qc = QuantumCircuit(2, 2)
state1 = [1,0]
state2 = [0,1]
qc.initialize(state1, 0)
qc.initialize(state2, 1)
qc.barrier()
qc.ry(theta, 0)
qc.barrier()
qc.cx(0, 1)
qc.barrier()
qc.measure(0, 0)
qc.measure(1, 1)


In [136]:
# drawing the circuit
qc.draw()

┌─────────────────┐ ░ ┌────────────┐ ░       ░ ┌─┐   
q_0: ┤ initialize(1,0) ├─░─┤ RY(param1) ├─░───■───░─┤M├───
     ├─────────────────┤ ░ └────────────┘ ░ ┌─┴─┐ ░ └╥┘┌─┐
q_1: ┤ initialize(0,1) ├─░────────────────░─┤ X ├─░──╫─┤M├
     └─────────────────┘ ░                ░ └───┘ ░  ║ └╥┘
c_0: ════════════════════════════════════════════════╩══╬═
                                                        ║ 
c_1: ═══════════════════════════════════════════════════╩═

In [69]:
# defining the function to execute the given number of shots on quantum simulator 

def execute_circuit(angle,shots):
    '''
        Parameters: angle - the initial random angle given as input
                    shots - number of shots to be executed on simulator
        Returns   : dictionary of possible states after execution with their respective counts
    '''
    rad = radians(angle) # conversion to degrees
    job = execute(qc, backend = Aer.get_backend('qasm_simulator'),shots = shots , parameter_binds=[{theta: rad}])
    counts = job.result().get_counts()
    return counts


In [70]:
def costfunction(prob_avg_01,prob_avg_10):
    '''
         Parameters: prob_avg_01 - average probability of getting the state 01 on measurement
                     prob_avg_10 - average probability of getting the state 10 on measurement
         Returns   : Mean Squared Error of the probabilities as a value to be improved with optimization techniques
    '''
    return pow((prob_avg_01 - prob_avg_10),2)
    

In [71]:
def costfunction_NAG(prob_avg_01,prob_avg_10,gama,V_t):
    '''
         Parameters: prob_avg_01 - average probability of getting the state 01 on measurement
                     prob_avg_10 - average probability of getting the state 10 on measurement
                     gama        - constant value parameter for Nesterov Accelerated Gradient
                     V_t         - using γV(t−1) in cost function for modifying the weights so that θ−γV(t−1) tells us the future location
         Returns   : Mean Squared Error of the probabilities as a value to be improved with optimization techniques
    '''
    return pow((prob_avg_01 - prob_avg_10 - gama*V_t),2)

In [72]:
# from tqdm import tqdm

In [129]:
init_angle = 120        # in degrees
no_of_shots = [1,10,100,1000]
learn_rates = [0.03,0.19,0.9,2]   # increasing learn rates for lesser time for convergence to optimum

for i in no_of_shots:
    print("Measurement counts for initial angle: ",str(init_angle)," and shots: ",str(i),"\t",str(execute_circuit(init_angle,i)))

Measurement counts for initial angle:  120  and shots:  1 	 {'10': 1}
Measurement counts for initial angle:  120  and shots:  10 	 {'10': 2, '01': 8}
Measurement counts for initial angle:  120  and shots:  100 	 {'10': 22, '01': 78}
Measurement counts for initial angle:  120  and shots:  1000 	 {'10': 222, '01': 778}


In [125]:
# classical gradient descent optimization step: θ=θ−α⋅∇J(θ)
#-----------------------------------------------
# learning_rate = 0.00001
max_i = 1000
init_angle = 120

for k,shots in enumerate(no_of_shots):
    learning_rate = learn_rates[k]
    angle = init_angle
    for i in range(max_i):
        counts = execute_circuit(angle,shots)
    
        try:
            prob_avg_01 = counts['01']/shots
        
        except:
            prob_avg_01 = 0
    
        try:
            prob_avg_10 = counts['10']/shots
        except:
            prob_avg_10 = 0
         
        angle = angle - learning_rate*(costfunction(prob_avg_01,prob_avg_10))
    print("Optimized parameter 'angle' in degrees: ",str(angle)," ; initial parameters, angle:",str(init_angle)," and shots: ",str(shots),"\t","Resultant state counts: ",str(counts))

Optimized parameter 'angle' in degrees:  89.99999999999886  ; initial parameters, angle: 120  and shots:  1 	 Resultant state counts:  {'10': 1}
Optimized parameter 'angle' in degrees:  90.71720000000147  ; initial parameters, angle: 120  and shots:  10 	 Resultant state counts:  {'10': 6, '01': 4}
Optimized parameter 'angle' in degrees:  89.05043999999889  ; initial parameters, angle: 120  and shots:  100 	 Resultant state counts:  {'10': 49, '01': 51}
Optimized parameter 'angle' in degrees:  90.8033839999995  ; initial parameters, angle: 120  and shots:  1000 	 Resultant state counts:  {'10': 472, '01': 528}


In [128]:
#  NAG optimization step: V(t)=γV(t−1)+α.∇J( θ−γV(t−1) )
#-----------------------------------------------
max_i = 1000
gama = 0.9
V_t = 0

for k,shots in enumerate(no_of_shots):
    learning_rate = learn_rates[k]
    angle = init_angle
    for i in range(max_i):
        counts = execute_circuit(angle,shots)
    
        try:
            prob_avg_01 = counts['01']/shots
        
        except:
            prob_avg_01 = 0
    
        try:
            prob_avg_10 = counts['10']/shots
        except:
            prob_avg_10 = 0
         
        V_T = gama*V_t + learning_rate*(costfunction_NAG(prob_avg_01,prob_avg_10,gama,V_t))
        angle = angle - V_T
        V_T = V_t
    print("Optimized parameter 'angle' in degrees: ",str(angle)," ; initial parameters, angle:",str(init_angle)," and shots: ",str(shots),"\t","Resultant state counts: ",str(counts))
    
    

Optimized parameter 'angle' in degrees:  89.99999999999886  ; initial parameters, angle: 120  and shots:  1 	 Resultant state counts:  {'01': 1}
Optimized parameter 'angle' in degrees:  90.55760000000163  ; initial parameters, angle: 120  and shots:  10 	 Resultant state counts:  {'10': 4, '01': 6}
Optimized parameter 'angle' in degrees:  88.83299999999915  ; initial parameters, angle: 120  and shots:  100 	 Resultant state counts:  {'10': 54, '01': 46}
Optimized parameter 'angle' in degrees:  90.75191999999939  ; initial parameters, angle: 120  and shots:  1000 	 Resultant state counts:  {'10': 498, '01': 502}
